# N Body Gravity Test

Some experiments to figure out the data structures for n body gravity sims

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import timeit

## Numpy Array Definitions

In [2]:
# Mass definitions
n = 10**3               # Number of bodies in disk
mE = 5.9722 * (10**24)  # Mass of Earth
rE = 6371.0 * (10**3)   # Radius of Earth
mDisk = 90              # Earth masses in disk
mass = (mE * mDisk) / n # Mass of single body

# Simulation constants
G = 6.67430 * (10**-11) # Gravitational constant
E = 1 * (10**-5)        # Softening factor (m), smaller when average distances are small

In [3]:
rng = np.random.default_rng()

masses = np.array([mass] * n)
velocities = rng.standard_normal((n, 3))
positions = rng.standard_normal((n, 3))

In [4]:
masses

array([5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23,
       5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23,
       5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23,
       5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23,
       5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23,
       5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23,
       5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23,
       5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23,
       5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23,
       5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23,
       5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23,
       5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23,
       5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23,
       5.37498e+23, 5.37498e+23, 5.37498e+23, 5.37498e+23, 5.374

In [5]:
velocities

array([[-0.08591575, -0.32754445,  0.49561605],
       [ 1.57221933,  2.05129058, -1.21922862],
       [ 1.97928968, -0.20138711,  1.38289044],
       ...,
       [-0.17091514, -1.30486624,  0.56438607],
       [-0.89573442, -1.14291246,  0.2659104 ],
       [ 0.5735328 , -0.65770716, -0.04500851]])

In [6]:
positions

array([[ 0.4607998 ,  0.1552256 , -1.11622677],
       [-0.3634172 ,  0.91983783, -1.85344464],
       [ 1.22226974, -0.37953245,  0.97380452],
       ...,
       [-0.14611701,  0.16271207,  1.78290236],
       [-0.7856306 , -1.63669791,  0.40927822],
       [-0.72954651,  1.36449166, -0.17962005]])

In [7]:
positions.dtype

dtype('float64')

## Array Memory Usage

In [8]:
bit32Mem = np.array(1.0, dtype = 'float32').nbytes
bit64Mem = np.array(1.0, dtype = 'float64').nbytes

In [9]:
print(f'32 bit: {bit32Mem * 7} B per object    {bit32Mem} B per value')
print(f'64 bit: {bit64Mem * 7} B per object    {bit64Mem} B per value')

32 bit: 28 B per object    4 B per value
64 bit: 56 B per object    8 B per value


In [10]:
mBytes = masses.nbytes
vBytes = velocities.nbytes
pBytes = positions.nbytes

In [11]:
print(f'Mass: {mBytes / 1024} KB')
print(f'Velocity: {vBytes / 1024} KB')
print(f'Position: {pBytes / 1024} KB')
print(f'Total: {(vBytes + pBytes + mBytes) / 1024} KB')

Mass: 7.8125 KB
Velocity: 23.4375 KB
Position: 23.4375 KB
Total: 54.6875 KB


## Find Acceleration

In [12]:
import numba
from numba import njit

In [13]:
# From github repo below, with minor modifications

def acc1(positions, masses, G, E):
    n = masses.size
    x = positions[:, 0:1]
    y = positions[:, 1:2]
    z = positions[:, 2:3]

    dx = x.T - x
    dy = y.T - y
    dz = z.T - z

    rInvCubed = (dx**2 + dy**2 + dz**2 + E**2)**(-1.5)

    ax = G * (dx * rInvCubed) @ masses
    ay = G * (dy * rInvCubed) @ masses
    az = G * (dz * rInvCubed) @ masses

    a = np.vstack((ax, ay, az)).T
    
    return a

In [14]:
@numba.jit()
def acc1numba(positions, masses, G, E):
    n = masses.size
    x = positions[:, 0:1]
    y = positions[:, 1:2]
    z = positions[:, 2:3]

    dx = x.T - x
    dy = y.T - y
    dz = z.T - z

    rInvCubed = (dx**2 + dy**2 + dz**2 + E**2)**(-1.5)

    ax = G * (dx * rInvCubed) @ masses
    ay = G * (dy * rInvCubed) @ masses
    az = G * (dz * rInvCubed) @ masses

    a = np.vstack((ax, ay, az)).T
    
    return a

In [15]:
def acc1blas(positions, masses, G, E):
    n = masses.size
    x = positions[:, 0:1]
    y = positions[:, 1:2]
    z = positions[:, 2:3]

    dx = x.T - x
    dy = y.T - y
    dz = z.T - z

    rInvCubed = (dx**2 + dy**2 + dz**2 + E**2)**(-1.5)

    ax = G * (dx * rInvCubed) @ masses
    ay = G * (dy * rInvCubed) @ masses
    az = G * (dz * rInvCubed) @ masses

    a = np.vstack((ax, ay, az)).T
    
    return a

In [16]:
import itertools
from scipy.linalg.blas import zhpr, dspr2, zhpmv

def acc2(positions, masses, G, E):
    n = masses.size
    # trick: use complex Hermitian to get the packed anti-symmetric
    # outer difference in the imaginary part of the zhpr answer
    # don't want to sum over dimensions yet, therefore must do them one-by-one
    trck = np.zeros((3, n * (n + 1) // 2), complex)
    for a, p in zip(trck, positions.T - 1j):
        zhpr(n, -2, p, a, 1, 0, 0, 1)
        # does  a  ->  a + alpha x x^H
        # parameters: n             --  matrix dimension
        #             alpha         --  real scalar
        #             x             --  complex vector
        #             ap            --  packed Hermitian n x n matrix a
        #                               i.e. an n(n+1)/2 vector
        #             incx          --  x stride
        #             offx          --  x offset
        #             lower         --  is storage of ap lower or upper
        #             overwrite_ap  --  whether to change a inplace
    # as a by-product we get positions positions^T:
    ppT = trck.real.sum(0) + 6
    # now compute matrix of squared distances ...
    # ... using (A-B)^2 = A^2 + B^2 - 2AB
    # ... that and the outer sum X (+) X.T equals X ones^T + ones X^T
    dspr2(n, -0.5, ppT[np.r_[0, 2:n+1].cumsum()], np.ones((n,)), ppT,
          1, 0, 1, 0, 0, 1)
    # does  a  ->  a + alpha x y^T + alpha y x^T    in packed symmetric storage
    # scale anti-symmetric differences by distance^-3
#     np.divide(trck.imag, ppT*np.sqrt(ppT), where=ppT.astype(bool),
#               out=trck.imag)
    np.divide(trck.imag, ((ppT**2 + E**2))**(3/4), where=ppT.astype(bool),
          out=trck.imag)
    # it remains to scale by massess and sum
    # this can be done by matrix multiplication with the vector of massesses ...
    # ... unfortunately because we need anti-symmetry we need to work
    # with Hermitian storage, i.e. complex numbers, even though the actual
    # computation is only real:
    out = np.zeros((3, n), complex)
    for a, o in zip(trck, out):
        zhpmv(n, 0.5, a, G * masses * -1j, 1, 0, 0, o, 1, 0, 0, 1)
        # multiplies packed Hermitian matrix by vector
    return out.real.T

In [17]:
from numpy.linalg import norm

def acc3(positions, masses, G, E):
    '''Params:
    - positions: numpy array of size (n,3)
    - masses: numpy array of size (n,)
    '''
    mass_matrix = masses.reshape((1, -1, 1))*masses.reshape((-1, 1, 1))
    disps = positions.reshape((1, -1, 3)) - positions.reshape((-1, 1, 3)) # displacements
    dists = norm(disps, axis=2)
    dists[dists == 0] = 1 # Avoid divide by zero warnings
    forces = G*disps*mass_matrix/np.expand_dims(dists, 2)**3
    return forces.sum(axis=1)/masses.reshape(-1, 1)

## Performance Comparison

In [18]:
%timeit acc1(positions, masses, G, E)

144 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [19]:
%timeit acc1blas(positions, masses, G, E)

137 ms ± 16.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%timeit acc1numba(positions, masses, G, E)

94.7 ms ± 5.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%timeit acc2(positions, masses, G, E)

48.4 ms ± 1.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
# %timeit acc3(positions, masses, G, E)

In [23]:
accelerations1 = acc1(positions, masses, G, E)
accelerations12 = acc1blas(positions, masses, G, E)
accelerations2 = acc2(positions, masses, G, E)
# accelerations3 = acc3(positions, masses, G, E)

In [24]:
accelerations1

array([[-3.03103216e+15, -1.06975333e+15,  8.62163952e+15],
       [ 8.99841482e+14, -2.48331724e+15,  5.45370309e+15],
       [-5.08439319e+15,  3.99895276e+15, -5.40219357e+15],
       ...,
       [ 1.06171393e+15,  2.45628480e+14, -7.27913392e+15],
       [ 3.23906418e+15,  5.80757060e+15, -1.68596661e+15],
       [ 4.43125479e+15, -6.25215298e+15,  3.46771704e+14]])

In [25]:
accelerations12

array([[-3.03103216e+15, -1.06975333e+15,  8.62163952e+15],
       [ 8.99841482e+14, -2.48331724e+15,  5.45370309e+15],
       [-5.08439319e+15,  3.99895276e+15, -5.40219357e+15],
       ...,
       [ 1.06171393e+15,  2.45628480e+14, -7.27913392e+15],
       [ 3.23906418e+15,  5.80757060e+15, -1.68596661e+15],
       [ 4.43125479e+15, -6.25215298e+15,  3.46771704e+14]])

In [26]:
accelerations2

array([[-3.03103215e+15, -1.06975328e+15,  8.62163950e+15],
       [ 8.99841485e+14, -2.48331724e+15,  5.45370309e+15],
       [-5.08439319e+15,  3.99895275e+15, -5.40219357e+15],
       ...,
       [ 1.06171393e+15,  2.45628476e+14, -7.27913391e+15],
       [ 3.23906418e+15,  5.80757060e+15, -1.68596661e+15],
       [ 4.43125477e+15, -6.25215297e+15,  3.46771704e+14]])

In [27]:
# accelerations3

## Sanity Test

In [28]:
E = 1 * (10**2)                 # Softening factor larger here
masses = np.array([mE, 1])      # Earth and a 1kg body
positions = np.array([
    [0, 0, 0],                  # Earth
    [rE, 0, 0]                  # Body on Earth surface
])

In [29]:
(G * mE) / rE**2

9.820302293385645

In [30]:
accelerations1 = acc1(positions, masses, G, E)
accelerations2 = acc2(positions, masses, G, E)
# accelerations3 = acc3(positions, masses, G, E)

In [31]:
accelerations1

array([[ 1.64433580e-24,  0.00000000e+00,  0.00000000e+00],
       [-9.82030229e+00,  0.00000000e+00,  0.00000000e+00]])

In [32]:
accelerations2

array([[ 1.64433580e-24,  0.00000000e+00,  0.00000000e+00],
       [-9.82030229e+00,  0.00000000e+00,  0.00000000e+00]])

In [33]:
# accelerations3

## References

- Nice Model: https://iopscience.iop.org/article/10.1088/0004-637X/716/2/1323
- Nvidia Implementation: https://developer.nvidia.com/gpugems/gpugems3/part-v-physics-simulation/chapter-31-fast-n-body-simulation-cuda
- Python N Body Implementation: https://github.com/pmocz/nbody-python/blob/master/nbody.py
- acc2: https://stackoverflow.com/a/52564537